In [ ]:
import pandas as pd
import numpy as np
import os
import wrds
from fredapi import Fred
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import warnings

root = '/Users/jingookwon/Dropbox/CapitalBudgetingHistory/Annual Reports'

In [ ]:
# Parameters
wrds_username = 'jingookwon'
wrds_password = 't47k!xXhvsaje!R'
fred_api_key = 'ccc8d4225575d294c70da7e99d782b17'

# Functions

## FF49 Industry Classification

In [ ]:
def ff49_fun(sic):
    """
    Map SIC codes to FF49 industry classifications

    Parameters:
    sic: SIC code; need to be int, float, or array-like

    Returns:
    int, float, or array-like
        FF49 industry classification (1-49) or NaN for unmatched SIC codes
    """
    if np.isscalar(sic):
        return _ff49_single(sic)
    else:
        sic_array = np.array(sic)
        result = np.full(sic_array.shape, np.nan)

        for i, s in enumerate(sic_array.flat):
            result.flat[i] = _ff49_single(s)
    
        return result

def _ff49_single(sic):
    """Helper function for single SIC code mapping"""
    if pd.isna(sic):
        return np.nan
    
    # Industry 1: Agriculture
    if (sic == 2048 or (100 <= sic <= 299) or (700 <= sic <= 799) or 
        (910 <= sic <= 919)):
        return 1
    
    # Industry 2: Food Products  
    elif (sic in [2095, 2098, 2099] or (2000 <= sic <= 2046) or 
          (2050 <= sic <= 2063) or (2070 <= sic <= 2079) or 
          (2090 <= sic <= 2092)):
        return 2
    
    # Industry 3: Candy & Soda
    elif (sic in [2086, 2087, 2096, 2097] or (2064 <= sic <= 2068)):
        return 3
    
    # Industry 4: Beer & Liquor
    elif (sic == 2080 or (2082 <= sic <= 2085)):
        return 4
    
    # Industry 5: Tobacco Products
    elif (2100 <= sic <= 2199):
        return 5
    
    # Industry 6: Recreation
    elif (sic in [3732, 3930, 3931] or (920 <= sic <= 999) or 
          (3650 <= sic <= 3652) or (3940 <= sic <= 3949)):
        return 6
    
    # Industry 7: Entertainment
    elif (sic in [7840, 7841, 7900, 7910, 7911, 7980] or 
          (7800 <= sic <= 7833) or (7920 <= sic <= 7933) or 
          (7940 <= sic <= 7949) or (7990 <= sic <= 7999)):
        return 7
    
    # Industry 8: Printing and Publishing
    elif (sic in [2770, 2771] or (2700 <= sic <= 2749) or 
          (2780 <= sic <= 2799)):
        return 8
    
    # Industry 9: Consumer Goods
    elif (sic in [2047, 2391, 2392, 3160, 3161, 3229, 3260, 3262, 3263, 
                  3269, 3230, 3231, 3750, 3751, 3800, 3860, 3861, 3910, 
                  3911, 3914, 3915, 3991, 3995] or 
          (2510 <= sic <= 2519) or (2590 <= sic <= 2599) or 
          (2840 <= sic <= 2844) or (3170 <= sic <= 3172) or 
          (3190 <= sic <= 3199) or (3630 <= sic <= 3639) or 
          (3870 <= sic <= 3873) or (3960 <= sic <= 3962)):
        return 9
    
    # Industry 10: Apparel
    elif (sic in [3020, 3021, 3130, 3131, 3150, 3151] or 
          (2300 <= sic <= 2390) or (3100 <= sic <= 3111) or 
          (3140 <= sic <= 3149) or (3963 <= sic <= 3965)):
        return 10
    
    # Industry 11: Healthcare
    elif (8000 <= sic <= 8099):
        return 11
    
    # Industry 12: Medical Equipment
    elif (sic in [3693, 3850, 3851] or (3840 <= sic <= 3849)):
        return 12
    
    # Industry 13: Pharmaceutical Products
    elif (sic in [2830, 2831] or (2833 <= sic <= 2836)):
        return 13
    
    # Industry 14: Chemicals
    elif ((2800 <= sic <= 2829) or (2850 <= sic <= 2879) or 
          (2890 <= sic <= 2899)):
        return 14
    
    # Industry 15: Rubber and Plastic Products
    elif (sic in [3031, 3041] or (3050 <= sic <= 3053) or 
          (3060 <= sic <= 3099)):
        return 15
    
    # Industry 16: Textiles
    elif ((2200 <= sic <= 2284) or (2290 <= sic <= 2295) or 
          (2297 <= sic <= 2299) or (2393 <= sic <= 2395) or 
          (2397 <= sic <= 2399)):
        return 16
    
    # Industry 17: Construction Materials
    elif (sic in [2660, 2661, 3200, 3210, 3211, 3240, 3241, 3261, 3264, 
                  3280, 3281, 3446, 3996] or 
          (800 <= sic <= 899) or (2400 <= sic <= 2439) or 
          (2450 <= sic <= 2459) or (2490 <= sic <= 2499) or 
          (2950 <= sic <= 2952) or (3250 <= sic <= 3259) or 
          (3270 <= sic <= 3275) or (3290 <= sic <= 3293) or 
          (3295 <= sic <= 3299) or (3420 <= sic <= 3429) or 
          (3430 <= sic <= 3433) or (3440 <= sic <= 3442) or 
          (3448 <= sic <= 3452) or (3490 <= sic <= 3499)):
        return 17
    
    # Industry 18: Construction
    elif ((1500 <= sic <= 1511) or (1520 <= sic <= 1549) or 
          (1600 <= sic <= 1799)):
        return 18
    
    # Industry 19: Steel Works Etc
    elif (sic == 3300 or (3310 <= sic <= 3317) or (3320 <= sic <= 3325) or 
          (3330 <= sic <= 3341) or (3350 <= sic <= 3357) or 
          (3360 <= sic <= 3379) or (3390 <= sic <= 3399)):
        return 19
    
    # Industry 20: Fabricated Products
    elif (sic in [3400, 3443, 3444] or (3460 <= sic <= 3479)):
        return 20
    
    # Industry 21: Machinery
    elif (sic in [3538, 3585, 3586] or (3510 <= sic <= 3536) or 
          (3540 <= sic <= 3569) or (3580 <= sic <= 3582) or 
          (3589 <= sic <= 3599)):
        return 21
    
    # Industry 22: Electrical Equipment
    elif (sic in [3600, 3620, 3621, 3648, 3649, 3660, 3699] or 
          (3610 <= sic <= 3613) or (3623 <= sic <= 3629) or 
          (3640 <= sic <= 3646) or (3690 <= sic <= 3692)):
        return 22
    
    # Industry 23: Automobiles and Trucks
    elif (sic in [2296, 2396, 3010, 3011, 3537, 3647, 3694, 3700, 3710, 
                  3711, 3799] or (3713 <= sic <= 3716) or 
          (3790 <= sic <= 3792)):
        return 23
    
    # Industry 24: Aircraft
    elif (sic in [3720, 3721, 3728, 3729] or (3723 <= sic <= 3725)):
        return 24
    
    # Industry 25: Shipbuilding, Railroad Equipment
    elif (sic in [3730, 3731] or (3740 <= sic <= 3743)):
        return 25
    
    # Industry 26: Defense
    elif (sic == 3795 or (3760 <= sic <= 3769) or (3480 <= sic <= 3489)):
        return 26
    
    # Industry 27: Precious Metals
    elif (1040 <= sic <= 1049):
        return 27
    
    # Industry 28: Non-Metallic and Industrial Metal Mining
    elif ((1000 <= sic <= 1039) or (1050 <= sic <= 1119) or 
          (1400 <= sic <= 1499)):
        return 28
    
    # Industry 29: Coal
    elif (1200 <= sic <= 1299):
        return 29
    
    # Industry 30: Petroleum and Natural Gas
    elif (sic in [1300, 1389] or (1310 <= sic <= 1339) or 
          (1370 <= sic <= 1382) or (2900 <= sic <= 2912) or 
          (2990 <= sic <= 2999)):
        return 30
    
    # Industry 31: Utilities
    elif (sic in [4900, 4910, 4911, 4939] or (4920 <= sic <= 4925) or 
          (4930 <= sic <= 4932) or (4940 <= sic <= 4942)):
        return 31
    
    # Industry 32: Communication
    elif (sic in [4800, 4899] or (4810 <= sic <= 4813) or 
          (4820 <= sic <= 4822) or (4830 <= sic <= 4841) or 
          (4880 <= sic <= 4892)):
        return 32
    
    # Industry 33: Personal Services
    elif (sic in [7020, 7021, 7200, 7230, 7231, 7240, 7241, 7250, 7251, 
                  7395, 7500, 7600, 7620, 7622, 7623, 7640, 7641] or 
          (7030 <= sic <= 7033) or (7210 <= sic <= 7212) or 
          (7214 <= sic <= 7217) or (7219 <= sic <= 7221) or 
          (7260 <= sic <= 7299) or (7520 <= sic <= 7549) or 
          (7629 <= sic <= 7631) or (7690 <= sic <= 7699) or 
          (8100 <= sic <= 8499) or (8600 <= sic <= 8699) or 
          (8800 <= sic <= 8899) or (7510 <= sic <= 7515)):
        return 33
    
    # Industry 34: Business Services
    elif (sic in [3993, 7218, 7300, 7374, 7396, 7397, 7399, 7519, 8700, 
                  8720, 8721] or (2750 <= sic <= 2759) or 
          (7310 <= sic <= 7342) or (7349 <= sic <= 7353) or 
          (7359 <= sic <= 7369) or (7376 <= sic <= 7385) or 
          (7389 <= sic <= 7394) or (8710 <= sic <= 8713) or 
          (8730 <= sic <= 8734) or (8740 <= sic <= 8748) or 
          (8900 <= sic <= 8911) or (8920 <= sic <= 8999) or 
          (4220 <= sic <= 4229)):
        return 34
    
    # Industry 35: Computers
    elif (sic == 3695 or (3570 <= sic <= 3579) or (3680 <= sic <= 3689)):
        return 35
    
    # Industry 36: Computer Software
    elif (sic == 7375 or (7370 <= sic <= 7373)):
        return 36
    
    # Industry 37: Electronic Equipment
    elif (sic in [3622, 3810, 3812] or (3661 <= sic <= 3666) or 
          (3669 <= sic <= 3679)):
        return 37
    
    # Industry 38: Measuring and Control Equipment
    elif (sic == 3811 or (3820 <= sic <= 3827) or (3829 <= sic <= 3839)):
        return 38
    
    # Industry 39: Business Supplies
    elif (sic in [2760, 2761] or (2520 <= sic <= 2549) or 
          (2600 <= sic <= 2639) or (2670 <= sic <= 2699) or 
          (3950 <= sic <= 3955)):
        return 39
    
    # Industry 40: Shipping Containers
    elif (sic in [3220, 3221] or (2440 <= sic <= 2449) or 
          (2640 <= sic <= 2659) or (3410 <= sic <= 3412)):
        return 40
    
    # Industry 41: Transportation
    elif (sic in [4100, 4130, 4131, 4150, 4151, 4230, 4231, 4780, 4789] or 
          (4000 <= sic <= 4013) or (4040 <= sic <= 4049) or 
          (4110 <= sic <= 4121) or (4140 <= sic <= 4142) or 
          (4170 <= sic <= 4173) or (4190 <= sic <= 4200) or 
          (4210 <= sic <= 4219) or (4240 <= sic <= 4249) or 
          (4400 <= sic <= 4700) or (4710 <= sic <= 4712) or 
          (4720 <= sic <= 4749) or (4782 <= sic <= 4785)):
        return 41
    
    # Industry 42: Wholesale
    elif (sic in [5000, 5099, 5100] or (5010 <= sic <= 5015) or 
          (5020 <= sic <= 5023) or (5030 <= sic <= 5060) or 
          (5063 <= sic <= 5065) or (5070 <= sic <= 5078) or 
          (5080 <= sic <= 5088) or (5090 <= sic <= 5094) or 
          (5110 <= sic <= 5113) or (5120 <= sic <= 5122) or 
          (5130 <= sic <= 5172) or (5180 <= sic <= 5182) or 
          (5190 <= sic <= 5199)):
        return 42
    
    # Industry 43: Retail
    elif (sic in [5200, 5250, 5251, 5260, 5261, 5270, 5271, 5300, 5310, 
                  5311, 5320, 5330, 5331, 5334, 5900, 5999] or 
          (5210 <= sic <= 5231) or (5340 <= sic <= 5349) or 
          (5390 <= sic <= 5400) or (5410 <= sic <= 5412) or 
          (5420 <= sic <= 5469) or (5490 <= sic <= 5500) or 
          (5510 <= sic <= 5579) or (5590 <= sic <= 5700) or 
          (5710 <= sic <= 5722) or (5730 <= sic <= 5736) or 
          (5750 <= sic <= 5799) or (5910 <= sic <= 5912) or 
          (5920 <= sic <= 5932) or (5940 <= sic <= 5990) or 
          (5992 <= sic <= 5995)):
        return 43
    
    # Industry 44: Restaurants, Hotels, Motels
    elif (sic in [7000, 7213] or (5800 <= sic <= 5829) or 
          (5890 <= sic <= 5899) or (7010 <= sic <= 7019) or 
          (7040 <= sic <= 7049)):
        return 44
    
    # Industry 45: Banking
    elif (sic == 6000 or (6010 <= sic <= 6036) or (6040 <= sic <= 6062) or 
          (6080 <= sic <= 6082) or (6090 <= sic <= 6100) or 
          (6110 <= sic <= 6113) or (6120 <= sic <= 6179) or 
          (6190 <= sic <= 6199)):
        return 45
    
    # Industry 46: Insurance
    elif (sic in [6300, 6350, 6351, 6360, 6361] or 
          (6310 <= sic <= 6331) or (6370 <= sic <= 6379) or 
          (6390 <= sic <= 6411)):
        return 46
    
    # Industry 47: Real Estate
    elif (sic in [6500, 6510, 6540, 6541, 6610, 6611] or 
          (6512 <= sic <= 6515) or (6517 <= sic <= 6532) or 
          (6550 <= sic <= 6553) or (6590 <= sic <= 6599)):
        return 47
    
    # Industry 48: Trading
    elif (sic in [6700, 6798, 6799] or (6200 <= sic <= 6299) or 
          (6710 <= sic <= 6726) or (6730 <= sic <= 6733) or 
          (6740 <= sic <= 6779) or (6790 <= sic <= 6795)):
        return 48
    
    # Industry 49: Almost Nothing
    elif (sic in [4970, 4971, 4990, 4991] or (4950 <= sic <= 4961)):
        return 49
    
    # Default case - unmatched SIC codes
    else:
        return np.nan

## Data Preparation

In [ ]:
def prepare_icc_data(daily_p, epsf, epsa, acc_g, acc_us, po_pref, bps, dps, rf10, ff49_ind, funda_g, funda):
    """
    Prepare data for ICC calculation

    Parameters:
    daily_p: pd.DataFrame
        Daily price data
    epsf: pd.DataFrame
        EPS forecasts from IBES
    epsa: pd.DataFrame
        EPS actuals from IBES
    acc_g: pd.DataFrame
        Global accounting data
    acc_us: pd.DataFrame
        US accounting data
    po_pref: str
        Preference for payout ratio source ('g_funda' or 'funda')
    bps: pd.DataFrame
        Book value per share data
    dps: pd.DataFrame
        Dividends per share data from IBES
    rf10: pd.DataFrame
        10-year risk-free data from FRED
    ff49_ind: pd.DataFrame
        FF49 industry classification
    funda_g: pd.DataFrame
        Global fundamental data
    funda: pd.DataFrame
        US fundamental data
    
    Returns:
    pd.DataFrame
        Prepared data for ICC calculation
    """

    # Processing EPS forecasts
    epsf_filtered = epsf[(epsf['measure'] == 'EPS') & (epsf['fpi'].isin([1,2]))].copy()
    epsf_filtered['fpi_label'] = 'eps' + epsf_filtered['fpi'].astype(str)

    # Pivot to wide
    epsf_st = epsf_filtered.pivot_table(
        index = ['id', 'datadate'],
        columns = 'fpi_label',
        values = 'value_adj',
        aggfunc = 'first'
    ).reset_index()

    # Fill missing eps2 with 15% growth assumption
    epsf_st['eps2'] = epsf_st['eps2'].fillna(epsf_st['eps1'] * 1.15)

    # Coverage calculation (12-month rolling average)
    numest_data = epsf[(epsf['measure'] == 'EPS') & (epsf['fpi'] == 1)][['id', 'datadate', 'numest']].copy()

    # Get unique dates and compute rolling coverage
    ibes_dates = sorted(numest_data['datadate'].unique())
    numest_list = []

    for i in range(11, len(ibes_dates)):
        start_date = ibes_dates[i - 11]
        end_date = ibes_dates[i]

        temp_data = numest_data[
            (numest_data['datadate'] >= start_date) & (numest_data['datadate'] <= end_date)
        ].groupby('id').agg({
            'numest': ['count', 'mean']
        }).reset_index()

        temp_data.columns = ['id', 'n', 'numest1_avg']
        temp_data = temp_data[temp_data['n'] == 12]
        temp_data['datadate'] = end_date
        numest_list.append(temp_data[['id', 'datadate', 'numest1_avg']])
    
    numest = pd.concat(numest_list, ignore_index=True)

    # LTG forecasts
    epsf_ltg = epsf[(epsf['measure'] == 'LTG') & (epsf['fpi'] == 0)].copy()
    epsf_ltg['ltg'] = epsf_ltg['medest'] / 100
    epsf_ltg = epsf_ltg[['id', 'datadate', 'ltg']]

    # Processing EPS actuals
    epsa_filtered = epsa[(epsa['measure'] == 'EPS') & (epsa['pdicity'] == 'ANN') 
                         & (epsa['value_adj'].notna()) & np.isfinite(epsa['value_adj'])].copy()
    epsa_ann = epsa_filtered[['id', 'datadate', 'value_adj']].copy()
    epsa_ann.columns = ['id', 'start', 'eps0']
    epsa_ann = epsa_ann.sort_values(['id', 'start'])

    # Create end dates
    epsa_ann['end'] = epsa_ann.groupby('id')['start'].shift(-1) - timedelta(days=1)
    epsa_ann['end'] = epsa_ann['end'].fillna(
        epsa_ann['start'] + relativedelta(months=13) - timedelta(days=1)
    )

    # DPS estimates from IBES
    dps_ib = dps[['id', 'datadate', 'value_adj']].copy()
    dps_ib.columns = ['id', 'start', 'ib_dps1']

    # Payout ratio from Compustat
    acc_g_subset = acc_g[['gvkey', 'datadate', 'at', 'dvt', 'ib']].copy()
    acc_g_subset['type'] = 'g_funda'

    acc_us_subset = acc_us[['gvkey', 'datadate', 'at', 'dvt', 'ib']].copy()
    acc_us_subset['type'] = 'funda'

    po_comp = pd.concat([acc_g_subset, acc_us_subset], ignore_index=True)
    po_comp = po_comp[po_comp['at'] > 0] # Non-negative assets

    # Compute payout ratio
    po_comp['po'] = np.where(
        po_comp['ib'] > 0, po_comp['dvt'] / po_comp['ib'], po_comp['dvt'] / (po_comp['at'] * 0.06)
    )

    # Handle duplicates
    po_comp['n'] = po_comp.groupby(['gvkey', 'datadate'])['gvkey'].transform('count')
    po_comp = po_comp[
        (po_comp['n'] == 1) | ((po_comp['n'] == 2) & (po_comp['type'] == po_pref))
    ].drop(['type', 'n'], axis=1)

    po_comp = po_comp.dropna(subset=['po']).sort_values(['gvkey', 'datadate'])

    # Add publication dates (4-month lag)
    po_comp['public_date'] = pd.to_datetime(po_comp['datadate']) + relativedelta(months=4)
    po_comp['start'] = po_comp['public_date']
    po_comp['end'] = po_comp.groupby('gvkey')['public_date'].shift(-1) - timedelta(days=1)
    po_comp['end'] = po_comp['end'].fillna(
        po_comp['start'] + relativedelta(years=1) - timedelta(days=1)
    )

    # Winsorize
    p01 = po_comp['po'].quantile(0.01)
    p99 = po_comp['po'].quantile(0.99)
    po_comp['po'] = po_comp['po'].clip(lower=p01, upper=p99)
    po_comp = po_comp[['gvkey', 'start', 'end', 'po']]

    # Industry ROE calculation
    funda_g_subset = funda_g[['gvkey', 'datadate', 'curcd', 'fx', 'ff49', 'seq', 'ib']].copy()
    funda_g_subset['int'] = True

    funda_subset = funda[['gvkey', 'datadate', 'curcd', 'fx', 'ff49', 'seq', 'ib']].copy()
    funda_subset['int'] = False

    acc = pd.concat([funda_g_subset, funda_subset], ignore_index=True)
    roe_comp = acc.sort_values(['gvkey', 'int', 'datadate']).copy()

    # Compute ROE
    roe_comp['roe'] = roe_comp.groupby(['gvkey', 'int'])['ib'] / roe_comp.groupby(['gvkey', 'int'])['seq'].shift(1)
    roe_comp['roe_usd'] = (roe_comp['ib'] * roe_comp['fx']) / (roe_comp.groupby(['gvkey', 'int'])['seq'].shift(1) * roe_comp.groupby(['gvkey', 'int'])['fx'].shift(1))

    # Currency changes
    roe_comp['curcd_lag'] = roe_comp.groupby(['gvkey', 'int'])['curcd'].shift(1)
    roe_comp.loc[roe_comp['curcd'] != roe_comp['curcd_lag'], 'roe'] = roe_comp['roe_usd']
    
    # Check year differences
    roe_comp['yr_diff'] = (roe_comp['datadate'] - roe_comp.groupby(['gvkey', 'int'])['datadate'].shift(1)).dt.days
    roe_comp = roe_comp[roe_comp['yr_diff'].between(365, 366, inclusive='both')]
    
    # Filter positive ROE and valid industry
    roe_comp = roe_comp[(roe_comp['roe'] > 0) & roe_comp['ff49'].notna()]
    
    # Handle duplicates (prefer international)
    roe_comp['n'] = roe_comp.groupby(['gvkey', 'datadate'])['gvkey'].transform('count')
    roe_comp = roe_comp[(roe_comp['n'] == 1) | ((roe_comp['n'] == 2) & (roe_comp['int'] == True))]
    
    # Calculate industry ROE (10-year rolling median)
    roe_comp['public_date'] = pd.to_datetime(roe_comp['datadate']) + relativedelta(months=4)
    
    unique_dates = sorted(epsf_st['datadate'].unique())
    roe_industry_list = []
    
    for date in unique_dates:
        date = pd.to_datetime(date)
        start_window = date - relativedelta(years=10)
        
        temp_roe = roe_comp[
            (roe_comp['public_date'] >= start_window) & 
            (roe_comp['public_date'] <= date)
        ].groupby('ff49')['roe'].agg(['count', 'median']).reset_index()
        
        temp_roe.columns = ['ff49', 'n', 'roe_ind']
        temp_roe['datadate'] = date
        roe_industry_list.append(temp_roe[['ff49', 'datadate', 'roe_ind']])
    
    roe_comp_final = pd.concat(roe_industry_list, ignore_index=True)
    
    # Winsorize industry ROE
    p01_roe = roe_comp_final['roe_ind'].quantile(0.01)
    p99_roe = roe_comp_final['roe_ind'].quantile(0.99)
    roe_comp_final['roe_ind'] = roe_comp_final['roe_ind'].clip(p01_roe, p99_roe)
    
    # Book-per-share data
    bps0 = bps[['id', 'datadate', 'bps_adj']].copy()
    bps0['start'] = pd.to_datetime(bps0['datadate']) + relativedelta(months=4)
    bps0['end'] = bps0.groupby('id')['start'].shift(-1) - timedelta(days=1)
    bps0['end'] = bps0['end'].fillna(bps0['start'] + relativedelta(months=12) - timedelta(days=1))
    bps0 = bps0.rename(columns={'bps_adj': 'bps0'})
    bps0 = bps0[['id', 'start', 'end', 'bps0']]
    
    # Combine data
    icc_data = pd.merge(epsf_st, daily_p, on=['id', 'datadate'], how='inner')
    
    # Screen: Require EPS1 and Price
    icc_data = icc_data[icc_data['prc_adj'].notna() & icc_data['eps1'].notna()]
    
    n_start = len(icc_data)
    
    # Add remaining data with left joins
    icc_data = pd.merge(icc_data, numest, on=['id', 'datadate'], how='left')
    icc_data = pd.merge(icc_data, epsf_ltg, on=['id', 'datadate'], how='left')
    icc_data = pd.merge(icc_data, dps_ib, on=['id', 'datadate'], how='left')
    
    # Time-based merges (equivalent to non-equi joins in data.table)
    def merge_time_range(left_df, right_df, left_key, right_key, date_col):
        """Helper function for time-range merges"""
        result_list = []
        for _, row in left_df.iterrows():
            merge_date = row[date_col]
            matching_rows = right_df[
                (right_df[right_key] == row[left_key]) &
                (right_df['start'] <= merge_date) &
                (right_df['end'] >= merge_date)
            ]
            if not matching_rows.empty:
                merged_row = row.to_dict()
                merged_row.update(matching_rows.iloc[0].to_dict())
                result_list.append(merged_row)
            else:
                merged_row = row.to_dict()
                # Add NaN values for missing columns
                for col in right_df.columns:
                    if col not in ['start', 'end', right_key]:
                        merged_row[col] = np.nan
                result_list.append(merged_row)
        
        return pd.DataFrame(result_list)
    
    # Merge payout ratios
    icc_data = merge_time_range(icc_data, po_comp, 'gvkey', 'gvkey', 'datadate')
    
    # Merge book value per share
    icc_data = merge_time_range(icc_data, bps0, 'id', 'id', 'datadate')
    
    # Merge EPS actuals
    icc_data = merge_time_range(icc_data, epsa_ann, 'id', 'id', 'datadate')
    
    # Merge risk-free rates
    rf10_temp = rf10.copy()
    rf10_temp = merge_time_range(icc_data[['datadate']], rf10_temp, 'datadate', 'datadate', 'datadate')
    icc_data = pd.merge(icc_data, rf10_temp[['datadate', 'rf10']], on='datadate', how='left')
    
    # Merge industry classifications
    icc_data = merge_time_range(icc_data, ff49_ind, 'gvkey', 'gvkey', 'datadate')
    
    # Merge industry ROE
    icc_data = pd.merge(icc_data, roe_comp_final, on=['ff49', 'datadate'], how='left')
    
    # Check for duplicates
    if len(icc_data) != n_start:
        warnings.warn("DATA HAS DUPLICATES")
    
    return icc_data


## Ohlson-Juettner (2005)

In [ ]:
def icc_oj_fun(data):
    """
    Calculate Ohlson-Juettner (2005) implied cost of capital.
    
    Parameters:
    -----------
    data : pd.DataFrame
        DataFrame containing required variables: eps1, eps2, ltg, po, 
        prc_adj, rf10, id, datadate
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with id, datadate, and icc_oj columns
    """
    # Filter for required variables (equivalent to data.table filtering)
    required_cols = ['eps1', 'eps2', 'ltg', 'po', 'prc_adj', 'rf10', 
                     'id', 'datadate']
    
    # Check if all required columns exist
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Filter rows where all required variables are not null
    oj = data.dropna(subset=['eps1', 'eps2', 'ltg', 'po'])[
        ['id', 'datadate', 'prc_adj', 'rf10', 'po', 'eps1', 'eps2', 'ltg']
    ].copy()
    
    # Risk premium calculation (rf10 minus 3% equity premium)
    oj['gam_min1'] = oj['rf10'] - 0.03
    
    # Intermediate calculation A
    oj['A'] = 0.5 * (oj['gam_min1'] + oj['eps1'] * oj['po'] / oj['prc_adj'])
    
    # Short-term growth rate calculation
    # Max ensures that STG >= LTG (see p. 450 of Mohanram and Gode)
    stg_calc = ((oj['eps2'] - oj['eps1']) / oj['eps1'] * oj['ltg']) ** 0.5
    oj['stg'] = np.maximum(stg_calc, oj['ltg'])
    
    # Ohlson-Juettner ICC calculation
    oj['icc_oj'] = (oj['A'] + 
                    np.sqrt(oj['A']**2 + 
                           oj['eps1'] / oj['prc_adj'] * 
                           (oj['stg'] - oj['gam_min1'])))
    
    # Return only required columns
    return oj[['id', 'datadate', 'icc_oj']]



In [1]:
import pandas as pd
import numpy as np

def icc_oj_fun(data):
    """
    Calculate Ohlson-Juettner (2005) implied cost of capital.
    
    Parameters:
    -----------
    data : pd.DataFrame
        DataFrame containing required variables: eps1, eps2, ltg, po, 
        prc_adj, rf10, id, datadate
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with id, datadate, and icc_oj columns
    """
    # Filter for required variables (equivalent to data.table filtering)
    required_cols = ['eps1', 'eps2', 'ltg', 'po', 'prc_adj', 'rf10', 
                     'id', 'datadate']
    
    # Check if all required columns exist
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Filter rows where all required variables are not null
    oj = data.dropna(subset=['eps1', 'eps2', 'ltg', 'po'])[
        ['id', 'datadate', 'prc_adj', 'rf10', 'po', 'eps1', 'eps2', 'ltg']
    ].copy()
    
    # Risk premium calculation (rf10 minus 3% equity premium)
    oj['gam_min1'] = oj['rf10'] - 0.03
    
    # Intermediate calculation A
    oj['A'] = 0.5 * (oj['gam_min1'] + oj['eps1'] * oj['po'] / oj['prc_adj'])
    
    # Short-term growth rate calculation
    # Max ensures that STG >= LTG (see p. 450 of Mohanram and Gode)
    stg_calc = ((oj['eps2'] - oj['eps1']) / oj['eps1'] * oj['ltg']) ** 0.5
    oj['stg'] = np.maximum(stg_calc, oj['ltg'])
    
    # Ohlson-Juettner ICC calculation
    oj['icc_oj'] = (oj['A'] + 
                    np.sqrt(oj['A']**2 + 
                           oj['eps1'] / oj['prc_adj'] * 
                           (oj['stg'] - oj['gam_min1'])))
    
    # Return only required columns
    return oj[['id', 'datadate', 'icc_oj']]


# Alternative version with more robust error handling
def icc_oj_fun_robust(data):
    """
    Calculate Ohlson-Juettner (2005) implied cost of capital with robust error handling.
    """
    # Required columns
    required_cols = ['eps1', 'eps2', 'ltg', 'po', 'prc_adj', 'rf10', 
                     'id', 'datadate']
    
    # Validate input
    if not isinstance(data, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame")
    
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Filter and copy data
    mask = (data['eps1'].notna() & 
            data['eps2'].notna() & 
            data['ltg'].notna() & 
            data['po'].notna() &
            data['prc_adj'].notna() & 
            data['rf10'].notna())
    
    if mask.sum() == 0:
        # Return empty DataFrame with correct structure
        return pd.DataFrame(columns=['id', 'datadate', 'icc_oj'])
    
    oj = data.loc[mask, required_cols].copy()
    
    # Handle potential division by zero
    if (oj['prc_adj'] == 0).any():
        print("Warning: Zero prices detected, filtering them out")
        oj = oj[oj['prc_adj'] != 0]
    
    if (oj['eps1'] == 0).any():
        print("Warning: Zero EPS1 detected, filtering them out")
        oj = oj[oj['eps1'] != 0]
    
    if len(oj) == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_oj'])
    
    # Calculations
    oj['gam_min1'] = oj['rf10'] - 0.03
    oj['A'] = 0.5 * (oj['gam_min1'] + oj['eps1'] * oj['po'] / oj['prc_adj'])
    
    # Short-term growth calculation with NaN handling
    growth_ratio = (oj['eps2'] - oj['eps1']) / oj['eps1']
    stg_calc = (growth_ratio * oj['ltg']) ** 0.5
    
    # Handle negative values under square root
    stg_calc = np.where(growth_ratio * oj['ltg'] >= 0, stg_calc, np.nan)
    oj['stg'] = np.maximum(stg_calc, oj['ltg'])
    
    # ICC calculation with error handling
    discriminant = (oj['A']**2 + 
                   oj['eps1'] / oj['prc_adj'] * (oj['stg'] - oj['gam_min1']))
    
    # Handle negative discriminant
    discriminant = np.maximum(discriminant, 0)
    
    oj['icc_oj'] = oj['A'] + np.sqrt(discriminant)
    
    # Filter out any remaining NaN or infinite results
    result = oj[['id', 'datadate', 'icc_oj']].copy()
    result = result[np.isfinite(result['icc_oj'])]
    
    return result


# Example usage and testing function
def test_icc_oj():
    """Test function with sample data"""
    # Create sample data
    np.random.seed(42)
    n = 1000
    
    test_data = pd.DataFrame({
        'id': range(n),
        'datadate': pd.date_range('2020-01-01', periods=n, freq='D'),
        'eps1': np.random.uniform(0.5, 3.0, n),
        'eps2': np.random.uniform(0.6, 3.5, n),
        'ltg': np.random.uniform(0.02, 0.15, n),
        'po': np.random.uniform(0.0, 0.8, n),
        'prc_adj': np.random.uniform(10, 100, n),
        'rf10': np.random.uniform(0.01, 0.05, n)
    })
    
    # Add some missing values to test handling
    test_data.loc[np.random.choice(n, 50, replace=False), 'eps1'] = np.nan
    test_data.loc[np.random.choice(n, 30, replace=False), 'ltg'] = np.nan
    
    # Test the function
    result = icc_oj_fun_robust(test_data)
    
    print(f"Input rows: {len(test_data)}")
    print(f"Output rows: {len(result)}")
    print(f"Average ICC_OJ: {result['icc_oj'].mean():.4f}")
    print(f"ICC_OJ range: {result['icc_oj'].min():.4f} to {result['icc_oj'].max():.4f}")
    
    return result

# Example usage
if __name__ == "__main__":
    # Run test
    test_result = test_icc_oj()
    print("\nFirst 5 results:")
    print(test_result.head())

Input rows: 1000
Output rows: 558
Average ICC_OJ: 0.0901
ICC_OJ range: 0.0244 to 0.2772

First 5 results:
   id   datadate    icc_oj
2   2 2020-01-03  0.088489
3   3 2020-01-04  0.129152
5   5 2020-01-06  0.112482
6   6 2020-01-07  0.096892
7   7 2020-01-08  0.090375


## PEG

In [2]:
def icc_peg_fun(data):
    """
    Calculate PEG (Easton 2004, Mohanram and Gode) implied cost of capital.
    
    Parameters:
    -----------
    data : pd.DataFrame
        DataFrame containing required variables: eps1, eps2, ltg, 
        prc_adj, id, datadate
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with id, datadate, and icc_peg columns
    """
    # Required columns for PEG model
    required_cols = ['eps1', 'eps2', 'ltg', 'prc_adj', 'id', 'datadate']
    
    # Check if all required columns exist
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Filter rows where required variables are not null
    peg = data.dropna(subset=['eps1', 'eps2', 'ltg'])[
        ['id', 'datadate', 'prc_adj', 'eps1', 'eps2', 'ltg']
    ].copy()
    
    # Short-term growth rate calculation
    # Max ensures that STG >= LTG (see p. 450 of Mohanram and Gode)
    stg_calc = ((peg['eps2'] - peg['eps1']) / peg['eps1'] * peg['ltg']) ** 0.5
    peg['stg'] = np.maximum(stg_calc, peg['ltg'])
    
    # PEG ICC calculation (simplified Easton model)
    peg['icc_peg'] = np.sqrt(peg['eps1'] / peg['prc_adj'] * peg['stg'])
    
    # Return only required columns
    return peg[['id', 'datadate', 'icc_peg']]


# Robust version with error handling
def icc_peg_fun_robust(data):
    """
    Calculate PEG implied cost of capital with robust error handling.
    """
    # Required columns
    required_cols = ['eps1', 'eps2', 'ltg', 'prc_adj', 'id', 'datadate']
    
    # Validate input
    if not isinstance(data, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame")
    
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Filter and copy data
    mask = (data['eps1'].notna() & 
            data['eps2'].notna() & 
            data['ltg'].notna() &
            data['prc_adj'].notna())
    
    if mask.sum() == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_peg'])
    
    peg = data.loc[mask, required_cols].copy()
    
    # Handle potential division by zero
    if (peg['prc_adj'] == 0).any():
        print("Warning: Zero prices detected, filtering them out")
        peg = peg[peg['prc_adj'] != 0]
    
    if (peg['eps1'] == 0).any():
        print("Warning: Zero EPS1 detected, filtering them out")
        peg = peg[peg['eps1'] != 0]
    
    if len(peg) == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_peg'])
    
    # Short-term growth calculation with error handling
    growth_ratio = (peg['eps2'] - peg['eps1']) / peg['eps1']
    stg_calc = (growth_ratio * peg['ltg']) ** 0.5
    
    # Handle negative values under square root
    stg_calc = np.where(growth_ratio * peg['ltg'] >= 0, stg_calc, np.nan)
    peg['stg'] = np.maximum(stg_calc, peg['ltg'])
    
    # PEG ICC calculation with error handling
    earnings_yield_growth = peg['eps1'] / peg['prc_adj'] * peg['stg']
    
    # Handle negative values under square root
    earnings_yield_growth = np.maximum(earnings_yield_growth, 0)
    
    peg['icc_peg'] = np.sqrt(earnings_yield_growth)
    
    # Filter out any remaining NaN or infinite results
    result = peg[['id', 'datadate', 'icc_peg']].copy()
    result = result[np.isfinite(result['icc_peg'])]
    
    return result


# Test function
def test_icc_peg():
    """Test function with sample data"""
    # Create sample data
    np.random.seed(42)
    n = 1000
    
    test_data = pd.DataFrame({
        'id': range(n),
        'datadate': pd.date_range('2020-01-01', periods=n, freq='D'),
        'eps1': np.random.uniform(0.5, 3.0, n),
        'eps2': np.random.uniform(0.6, 3.5, n),
        'ltg': np.random.uniform(0.02, 0.15, n),
        'prc_adj': np.random.uniform(10, 100, n)
    })
    
    # Add some missing values to test handling
    test_data.loc[np.random.choice(n, 50, replace=False), 'eps1'] = np.nan
    test_data.loc[np.random.choice(n, 30, replace=False), 'ltg'] = np.nan
    
    # Test the function
    result = icc_peg_fun_robust(test_data)
    
    print(f"Input rows: {len(test_data)}")
    print(f"Output rows: {len(result)}")
    print(f"Average ICC_PEG: {result['icc_peg'].mean():.4f}")
    print(f"ICC_PEG range: {result['icc_peg'].min():.4f} to {result['icc_peg'].max():.4f}")
    
    return result


# Example usage
if __name__ == "__main__":
    # Run test
    test_result = test_icc_peg()
    print("\nFirst 5 results:")
    print(test_result.head())

Input rows: 1000
Output rows: 551
Average ICC_PEG: 0.0837
ICC_PEG range: 0.0212 to 0.2345

First 5 results:
   id   datadate   icc_peg
2   2 2020-01-03  0.124860
3   3 2020-01-04  0.064534
4   4 2020-01-05  0.071901
5   5 2020-01-06  0.069790
6   6 2020-01-07  0.058677


## Gebhardt-Lee-Swaminathan (2001)

In [3]:
def icc_gls_fun(data, root_low=0.01, root_high=0.5):
    """
    Calculate Gebhardt, Lee, and Swaminathan (2001) implied cost of capital.
    
    Parameters:
    -----------
    data : pd.DataFrame
        DataFrame containing required variables: eps1, eps2, bps0, prc_adj, 
        po, roe_ind, id, datadate
    root_low : float, default=0.01
        Lower bound for root finding
    root_high : float, default=0.5
        Upper bound for root finding
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with id, datadate, and icc_gls columns
    """
    
    # Required columns
    required_cols = ['eps1', 'eps2', 'bps0', 'prc_adj', 'po', 'roe_ind', 
                     'id', 'datadate']
    
    # Validate input
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Filter and copy data
    mask = (data['eps1'].notna() & 
            data['eps2'].notna() & 
            data['bps0'].notna() &
            data['prc_adj'].notna() & 
            data['po'].notna() &
            data['roe_ind'].notna())
    
    if mask.sum() == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_gls'])
    
    gls = data.loc[mask, required_cols].copy()
    
    # Additional filtering for infinite values and positive book values
    gls = gls[np.isfinite(gls['eps1']) & 
              np.isfinite(gls['bps0']) & 
              (gls['bps0'] > 0)]
    
    if len(gls) == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_gls'])
    
    # Forward projections for years 1-2
    gls['bps1'] = gls['bps0'] + gls['eps1'] * (1 - gls['po'])
    gls['bps2'] = gls['bps1'] + gls['eps2'] * (1 - gls['po'])
    gls['roe1'] = gls['eps1'] / gls['bps0']
    gls['roe2'] = gls['eps2'] / gls['bps1']
    
    # Linear convergence parameter to industry ROE over 10 years
    gls['g'] = (gls['roe_ind'] - gls['roe2']) / 10
    
    # Project years 3-12
    for h in range(3, 13):  # 3 to 12 inclusive
        # ROE converges linearly to industry average
        gls[f'roe{h}'] = gls['roe2'] + gls['g'] * (h - 2)
        
        # Book value grows with retained earnings
        gls[f'bps{h}'] = gls[f'bps{h-1}'] * (1 + gls[f'roe{h}'] * (1 - gls['po']))
    
    def gls_func(r, p, bps0, bps1, bps2, bps3, bps4, bps5, bps6, bps7, 
                 bps8, bps9, bps10, bps11, roe1, roe2, roe3, roe4, roe5, 
                 roe6, roe7, roe8, roe9, roe10, roe11, roe12):
        """
        Residual income valuation function.
        Returns price minus theoretical value (should be zero at correct r).
        """
        residual_income = (
            ((roe1 - r) * bps0) / (1 + r)**1 +
            ((roe2 - r) * bps1) / (1 + r)**2 +
            ((roe3 - r) * bps2) / (1 + r)**3 +
            ((roe4 - r) * bps3) / (1 + r)**4 +
            ((roe5 - r) * bps4) / (1 + r)**5 +
            ((roe6 - r) * bps5) / (1 + r)**6 +
            ((roe7 - r) * bps6) / (1 + r)**7 +
            ((roe8 - r) * bps7) / (1 + r)**8 +
            ((roe9 - r) * bps8) / (1 + r)**9 +
            ((roe10 - r) * bps9) / (1 + r)**10 +
            ((roe11 - r) * bps10) / (1 + r)**11 +  # FIXED: was bps11
            ((roe12 - r) * bps11) / (r * (1 + r)**11)  # Terminal value
        )
        
        theoretical_price = bps0 + residual_income
        return p - theoretical_price
    
    # Check interval validity and solve
    results = []
    
    for idx, row in gls.iterrows():
        try:
            # Test interval bounds
            int_low = gls_func(
                r=root_low, p=row['prc_adj'],
                bps0=row['bps0'], bps1=row['bps1'], bps2=row['bps2'], 
                bps3=row['bps3'], bps4=row['bps4'], bps5=row['bps5'],
                bps6=row['bps6'], bps7=row['bps7'], bps8=row['bps8'], 
                bps9=row['bps9'], bps10=row['bps10'], bps11=row['bps11'],
                roe1=row['roe1'], roe2=row['roe2'], roe3=row['roe3'], 
                roe4=row['roe4'], roe5=row['roe5'], roe6=row['roe6'],
                roe7=row['roe7'], roe8=row['roe8'], roe9=row['roe9'], 
                roe10=row['roe10'], roe11=row['roe11'], roe12=row['roe12']
            )
            
            int_high = gls_func(
                r=root_high, p=row['prc_adj'],
                bps0=row['bps0'], bps1=row['bps1'], bps2=row['bps2'], 
                bps3=row['bps3'], bps4=row['bps4'], bps5=row['bps5'],
                bps6=row['bps6'], bps7=row['bps7'], bps8=row['bps8'], 
                bps9=row['bps9'], bps10=row['bps10'], bps11=row['bps11'],
                roe1=row['roe1'], roe2=row['roe2'], roe3=row['roe3'], 
                roe4=row['roe4'], roe5=row['roe5'], roe6=row['roe6'],
                roe7=row['roe7'], roe8=row['roe8'], roe9=row['roe9'], 
                roe10=row['roe10'], roe11=row['roe11'], roe12=row['roe12']
            )
            
            # Check if root exists in interval (opposite signs)
            if np.sign(int_low) != np.sign(int_high) and not np.isnan(int_low):
                # Solve for ICC using scipy's brentq (more robust than uniroot)
                from scipy.optimize import brentq
                
                def objective(r):
                    return gls_func(
                        r=r, p=row['prc_adj'],
                        bps0=row['bps0'], bps1=row['bps1'], bps2=row['bps2'], 
                        bps3=row['bps3'], bps4=row['bps4'], bps5=row['bps5'],
                        bps6=row['bps6'], bps7=row['bps7'], bps8=row['bps8'], 
                        bps9=row['bps9'], bps10=row['bps10'], bps11=row['bps11'],
                        roe1=row['roe1'], roe2=row['roe2'], roe3=row['roe3'], 
                        roe4=row['roe4'], roe5=row['roe5'], roe6=row['roe6'],
                        roe7=row['roe7'], roe8=row['roe8'], roe9=row['roe9'], 
                        roe10=row['roe10'], roe11=row['roe11'], roe12=row['roe12']
                    )
                
                icc_value = brentq(objective, root_low, root_high)
                
                results.append({
                    'id': row['id'],
                    'datadate': row['datadate'],
                    'icc_gls': icc_value
                })
            
        except Exception as e:
            # Skip problematic cases
            continue
    
    return pd.DataFrame(results)


# Robust version with vectorized operations (faster for large datasets)
def icc_gls_fun_vectorized(data, root_low=0.01, root_high=0.5, max_workers=4):
    """
    Vectorized version using parallel processing for better performance.
    """
    import concurrent.futures
    from scipy.optimize import brentq
    
    # Same initial setup as above...
    required_cols = ['eps1', 'eps2', 'bps0', 'prc_adj', 'po', 'roe_ind', 
                     'id', 'datadate']
    
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    mask = (data['eps1'].notna() & data['eps2'].notna() & 
            data['bps0'].notna() & data['prc_adj'].notna() & 
            data['po'].notna() & data['roe_ind'].notna())
    
    if mask.sum() == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_gls'])
    
    gls = data.loc[mask, required_cols].copy()
    gls = gls[np.isfinite(gls['eps1']) & 
              np.isfinite(gls['bps0']) & 
              (gls['bps0'] > 0)]
    
    if len(gls) == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_gls'])
    
    # Same projections...
    gls['bps1'] = gls['bps0'] + gls['eps1'] * (1 - gls['po'])
    gls['bps2'] = gls['bps1'] + gls['eps2'] * (1 - gls['po'])
    gls['roe1'] = gls['eps1'] / gls['bps0']
    gls['roe2'] = gls['eps2'] / gls['bps1']
    gls['g'] = (gls['roe_ind'] - gls['roe2']) / 10
    
    for h in range(3, 13):
        gls[f'roe{h}'] = gls['roe2'] + gls['g'] * (h - 2)
        gls[f'bps{h}'] = gls[f'bps{h-1}'] * (1 + gls[f'roe{h}'] * (1 - gls['po']))
    
    def solve_single_row(row_tuple):
        """Helper function for parallel processing"""
        idx, row = row_tuple
        try:
            def objective(r):
                residual_income = (
                    ((row['roe1'] - r) * row['bps0']) / (1 + r)**1 +
                    ((row['roe2'] - r) * row['bps1']) / (1 + r)**2 +
                    ((row['roe3'] - r) * row['bps2']) / (1 + r)**3 +
                    ((row['roe4'] - r) * row['bps3']) / (1 + r)**4 +
                    ((row['roe5'] - r) * row['bps4']) / (1 + r)**5 +
                    ((row['roe6'] - r) * row['bps5']) / (1 + r)**6 +
                    ((row['roe7'] - r) * row['bps6']) / (1 + r)**7 +
                    ((row['roe8'] - r) * row['bps7']) / (1 + r)**8 +
                    ((row['roe9'] - r) * row['bps8']) / (1 + r)**9 +
                    ((row['roe10'] - r) * row['bps9']) / (1 + r)**10 +
                    ((row['roe11'] - r) * row['bps10']) / (1 + r)**11 +
                    ((row['roe12'] - r) * row['bps11']) / (r * (1 + r)**11)
                )
                return row['prc_adj'] - (row['bps0'] + residual_income)
            
            # Test bounds
            if np.sign(objective(root_low)) != np.sign(objective(root_high)):
                icc_value = brentq(objective, root_low, root_high)
                return {
                    'id': row['id'],
                    'datadate': row['datadate'],
                    'icc_gls': icc_value
                }
        except:
            pass
        return None
    
    # Parallel processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(solve_single_row, gls.iterrows()))
    
    # Filter out None results
    results = [r for r in results if r is not None]
    
    return pd.DataFrame(results)


# Test function
def test_icc_gls():
    """Test function with sample data"""
    np.random.seed(42)
    n = 100
    
    test_data = pd.DataFrame({
        'id': range(n),
        'datadate': pd.date_range('2020-01-01', periods=n, freq='D'),
        'eps1': np.random.uniform(0.5, 3.0, n),
        'eps2': np.random.uniform(0.6, 3.5, n),
        'bps0': np.random.uniform(5, 50, n),
        'prc_adj': np.random.uniform(10, 100, n),
        'po': np.random.uniform(0.0, 0.8, n),
        'roe_ind': np.random.uniform(0.05, 0.25, n)
    })
    
    # Test the function
    result = icc_gls_fun(test_data)
    
    print(f"Input rows: {len(test_data)}")
    print(f"Output rows: {len(result)}")
    if len(result) > 0:
        print(f"Average ICC_GLS: {result['icc_gls'].mean():.4f}")
        print(f"ICC_GLS range: {result['icc_gls'].min():.4f} to {result['icc_gls'].max():.4f}")
    
    return result


if __name__ == "__main__":
    # Run test
    test_result = test_icc_gls()
    if len(test_result) > 0:
        print("\nFirst 5 results:")
        print(test_result.head())

Input rows: 100
Output rows: 98
Average ICC_GLS: 0.0971
ICC_GLS range: 0.0114 to 0.3091

First 5 results:
   id   datadate   icc_gls
0   0 2020-01-01  0.182016
1   1 2020-01-02  0.041466
2   2 2020-01-03  0.037690
3   3 2020-01-04  0.105928
4   4 2020-01-05  0.093460


## Claus-Thomas (2001)

In [4]:
import pandas as pd
import numpy as np
from scipy.optimize import brentq
import warnings

def icc_ct_fun(data, root_low=0.01, root_high=0.5):
    """
    Calculate Claus and Thomas (2001) implied cost of capital.
    
    Parameters:
    -----------
    data : pd.DataFrame
        DataFrame containing required variables: eps1, eps2, ltg, bps0, 
        prc_adj, po, roe_ind, rf10, id, datadate
    root_low : float, default=0.01
        Lower bound for root finding
    root_high : float, default=0.5
        Upper bound for root finding
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with id, datadate, and icc_ct columns
    """
    
    # Required columns
    required_cols = ['eps1', 'eps2', 'ltg', 'bps0', 'prc_adj', 'po', 
                     'roe_ind', 'rf10', 'id', 'datadate']
    
    # Validate input
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Filter and copy data
    mask = (data['eps1'].notna() & 
            data['eps2'].notna() & 
            data['ltg'].notna() &
            data['bps0'].notna() &
            data['prc_adj'].notna() & 
            data['po'].notna() &
            data['roe_ind'].notna() &
            data['rf10'].notna())
    
    if mask.sum() == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_ct'])
    
    ct = data.loc[mask, required_cols].copy()
    
    # Additional filtering for infinite values and positive book values
    ct = ct[np.isfinite(ct['eps1']) & 
            np.isfinite(ct['bps0']) & 
            (ct['bps0'] > 0)]
    
    if len(ct) == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_ct'])
    
    # Calculate risk premium (g)
    ct['g'] = ct['rf10'] - 0.03
    
    # Forward projections for years 1-2
    ct['bps1'] = ct['bps0'] + ct['eps1'] * (1 - ct['po'])
    ct['bps2'] = ct['bps1'] + ct['eps2'] * (1 - ct['po'])
    ct['roe1'] = ct['eps1'] / ct['bps0']
    ct['roe2'] = ct['eps2'] / ct['bps1']
    
    # Project years 3-5 with constant LTG growth
    for h in range(3, 6):  # 3 to 5 inclusive
        # EPS grows at LTG rate
        ct[f'eps{h}'] = ct[f'eps{h-1}'] * (1 + ct['ltg'])
        
        # ROE calculation
        ct[f'roe{h}'] = ct[f'eps{h}'] / ct[f'bps{h-1}']
        
        # Book value evolution
        ct[f'bps{h}'] = ct[f'bps{h-1}'] + ct[f'eps{h}'] * (1 - ct['po'])
    
    def ct_func(r, p, g, bps0, bps1, bps2, bps3, bps4, 
                roe1, roe2, roe3, roe4, roe5):
        """
        CT residual income valuation function.
        Returns price minus theoretical value (should be zero at correct r).
        """
        # Residual income for years 1-5
        residual_income = (
            ((roe1 - r) * bps0) / (1 + r)**1 +
            ((roe2 - r) * bps1) / (1 + r)**2 +
            ((roe3 - r) * bps2) / (1 + r)**3 +
            ((roe4 - r) * bps3) / (1 + r)**4 +
            ((roe5 - r) * bps4) / (1 + r)**5
        )
        
        # Terminal value (growing perpetuity from year 6)
        # Avoid division by zero when r ≈ g
        if abs(r - g) < 1e-10:
            terminal_value = 0
        else:
            terminal_value = ((roe5 - r) * (1 + g) * bps4) / ((r - g) * (1 + r)**5)
        
        theoretical_price = bps0 + residual_income + terminal_value
        return p - theoretical_price
    
    # Check interval validity and solve
    results = []
    
    for idx, row in ct.iterrows():
        try:
            # Ensure r > g to avoid terminal value singularity
            r_low = max(root_low, row['g'] + 0.001)
            
            # Test interval bounds
            int_low = ct_func(
                r=r_low, p=row['prc_adj'], g=row['g'],
                bps0=row['bps0'], bps1=row['bps1'], bps2=row['bps2'], 
                bps3=row['bps3'], bps4=row['bps4'],
                roe1=row['roe1'], roe2=row['roe2'], roe3=row['roe3'], 
                roe4=row['roe4'], roe5=row['roe5']
            )
            
            int_high = ct_func(
                r=root_high, p=row['prc_adj'], g=row['g'],
                bps0=row['bps0'], bps1=row['bps1'], bps2=row['bps2'], 
                bps3=row['bps3'], bps4=row['bps4'],
                roe1=row['roe1'], roe2=row['roe2'], roe3=row['roe3'], 
                roe4=row['roe4'], roe5=row['roe5']
            )
            
            # Check if root exists in interval (opposite signs)
            if (np.sign(int_low) != np.sign(int_high) and 
                not np.isnan(int_low) and not np.isnan(int_high)):
                
                # Solve for ICC
                def objective(r):
                    return ct_func(
                        r=r, p=row['prc_adj'], g=row['g'],
                        bps0=row['bps0'], bps1=row['bps1'], bps2=row['bps2'], 
                        bps3=row['bps3'], bps4=row['bps4'],
                        roe1=row['roe1'], roe2=row['roe2'], roe3=row['roe3'], 
                        roe4=row['roe4'], roe5=row['roe5']
                    )
                
                icc_value = brentq(objective, r_low, root_high)
                
                results.append({
                    'id': row['id'],
                    'datadate': row['datadate'],
                    'icc_ct': icc_value
                })
            
        except Exception as e:
            # Skip problematic cases
            continue
    
    return pd.DataFrame(results)


# Vectorized version for better performance with large datasets
def icc_ct_fun_vectorized(data, root_low=0.01, root_high=0.5, max_workers=4):
    """
    Vectorized version using parallel processing for better performance.
    """
    import concurrent.futures
    
    # Same initial setup as above...
    required_cols = ['eps1', 'eps2', 'ltg', 'bps0', 'prc_adj', 'po', 
                     'roe_ind', 'rf10', 'id', 'datadate']
    
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    mask = (data['eps1'].notna() & data['eps2'].notna() & 
            data['ltg'].notna() & data['bps0'].notna() & 
            data['prc_adj'].notna() & data['po'].notna() & 
            data['roe_ind'].notna() & data['rf10'].notna())
    
    if mask.sum() == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_ct'])
    
    ct = data.loc[mask, required_cols].copy()
    ct = ct[np.isfinite(ct['eps1']) & 
            np.isfinite(ct['bps0']) & 
            (ct['bps0'] > 0)]
    
    if len(ct) == 0:
        return pd.DataFrame(columns=['id', 'datadate', 'icc_ct'])
    
    # Same projections...
    ct['g'] = ct['rf10'] - 0.03
    ct['bps1'] = ct['bps0'] + ct['eps1'] * (1 - ct['po'])
    ct['bps2'] = ct['bps1'] + ct['eps2'] * (1 - ct['po'])
    ct['roe1'] = ct['eps1'] / ct['bps0']
    ct['roe2'] = ct['eps2'] / ct['bps1']
    
    for h in range(3, 6):
        ct[f'eps{h}'] = ct[f'eps{h-1}'] * (1 + ct['ltg'])
        ct[f'roe{h}'] = ct[f'eps{h}'] / ct[f'bps{h-1}']
        ct[f'bps{h}'] = ct[f'bps{h-1}'] + ct[f'eps{h}'] * (1 - ct['po'])
    
    def solve_single_row(row_tuple):
        """Helper function for parallel processing"""
        idx, row = row_tuple
        try:
            def objective(r):
                # Residual income for years 1-5
                residual_income = (
                    ((row['roe1'] - r) * row['bps0']) / (1 + r)**1 +
                    ((row['roe2'] - r) * row['bps1']) / (1 + r)**2 +
                    ((row['roe3'] - r) * row['bps2']) / (1 + r)**3 +
                    ((row['roe4'] - r) * row['bps3']) / (1 + r)**4 +
                    ((row['roe5'] - r) * row['bps4']) / (1 + r)**5
                )
                
                # Terminal value
                if abs(r - row['g']) < 1e-10:
                    terminal_value = 0
                else:
                    terminal_value = ((row['roe5'] - r) * (1 + row['g']) * row['bps4']) / ((r - row['g']) * (1 + r)**5)
                
                return row['prc_adj'] - (row['bps0'] + residual_income + terminal_value)
            
            # Test bounds
            r_low = max(root_low, row['g'] + 0.001)
            
            if np.sign(objective(r_low)) != np.sign(objective(root_high)):
                icc_value = brentq(objective, r_low, root_high)
                return {
                    'id': row['id'],
                    'datadate': row['datadate'],
                    'icc_ct': icc_value
                }
        except:
            pass
        return None
    
    # Parallel processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(solve_single_row, ct.iterrows()))
    
    # Filter out None results
    results = [r for r in results if r is not None]
    
    return pd.DataFrame(results)


# Test function
def test_icc_ct():
    """Test function with sample data"""
    np.random.seed(42)
    n = 100
    
    test_data = pd.DataFrame({
        'id': range(n),
        'datadate': pd.date_range('2020-01-01', periods=n, freq='D'),
        'eps1': np.random.uniform(0.5, 3.0, n),
        'eps2': np.random.uniform(0.6, 3.5, n),
        'ltg': np.random.uniform(0.02, 0.15, n),  # 2% to 15% growth
        'bps0': np.random.uniform(5, 50, n),
        'prc_adj': np.random.uniform(10, 100, n),
        'po': np.random.uniform(0.0, 0.8, n),
        'roe_ind': np.random.uniform(0.05, 0.25, n),
        'rf10': np.random.uniform(0.01, 0.05, n)  # 1% to 5% risk-free rate
    })
    
    # Test the function
    result = icc_ct_fun(test_data)
    
    print(f"Input rows: {len(test_data)}")
    print(f"Output rows: {len(result)}")
    if len(result) > 0:
        print(f"Average ICC_CT: {result['icc_ct'].mean():.4f}")
        print(f"ICC_CT range: {result['icc_ct'].min():.4f} to {result['icc_ct'].max():.4f}")
        print(f"Success rate: {len(result)/len(test_data):.2%}")
    
    return result


# Comparison function to show differences between CT and other methods
def compare_icc_methods():
    """Compare CT model with simplified versions"""
    np.random.seed(42)
    n = 50
    
    test_data = pd.DataFrame({
        'id': range(n),
        'datadate': pd.date_range('2020-01-01', periods=n, freq='D'),
        'eps1': np.random.uniform(1.0, 3.0, n),
        'eps2': np.random.uniform(1.2, 3.5, n),
        'ltg': np.random.uniform(0.05, 0.12, n),
        'bps0': np.random.uniform(10, 30, n),
        'prc_adj': np.random.uniform(20, 80, n),
        'po': np.random.uniform(0.2, 0.6, n),
        'roe_ind': np.random.uniform(0.08, 0.18, n),
        'rf10': np.random.uniform(0.02, 0.04, n)
    })
    
    # Calculate CT ICC
    ct_result = icc_ct_fun(test_data)
    
    print("CT Model Characteristics:")
    print(f"- Uses 5-year explicit forecasting period")
    print(f"- Terminal value with growing perpetuity")
    print(f"- Risk premium g = rf10 - 3%")
    print(f"- Automatic constraint handling (r > g)")
    print(f"\nResults: {len(ct_result)} successful calculations out of {len(test_data)}")
    
    if len(ct_result) > 0:
        print(f"Mean ICC: {ct_result['icc_ct'].mean():.4f}")
        print(f"Std ICC: {ct_result['icc_ct'].std():.4f}")
    
    return ct_result


if __name__ == "__main__":
    # Run tests
    print("Testing CT ICC Function:")
    print("=" * 50)
    test_result = test_icc_ct()
    
    print("\n" + "=" * 50)
    print("Comparing ICC Methods:")
    print("=" * 50)
    comparison_result = compare_icc_methods()
    
    if len(test_result) > 0:
        print("\nFirst 5 CT results:")
        print(test_result.head())

Testing CT ICC Function:
Input rows: 100
Output rows: 97
Average ICC_CT: 0.0601
ICC_CT range: 0.0107 to 0.2422
Success rate: 97.00%

Comparing ICC Methods:
CT Model Characteristics:
- Uses 5-year explicit forecasting period
- Terminal value with growing perpetuity
- Risk premium g = rf10 - 3%
- Automatic constraint handling (r > g)

Results: 50 successful calculations out of 50
Mean ICC: 0.0580
Std ICC: 0.0275

First 5 CT results:
   id   datadate    icc_ct
0   0 2020-01-01  0.045435
1   1 2020-01-02  0.017534
2   2 2020-01-03  0.025867
3   3 2020-01-04  0.026378
4   4 2020-01-05  0.088572
